In [412]:
import pandas as pd
import numpy as np

In [413]:
### DEFINE BETA VALUES FOR VDF

beta_vals = {} # TBD: compute values based on growth

beta_template_aux = {
    "Period": ["AM", "MD", "PM", "NT"],
    1: [3.246, 3.249, 3.399, 3.402],
    2: [0.715, 0.708, 0.592, 0.708],
    3: [3.130, 3.133, 3.283, 3.286]
}

beta_vals_aux = pd.DataFrame(beta_template_aux)

beta_vals_aux.set_index("Period", inplace=True)

beta_vals[2025] = beta_vals_aux

In [414]:
### DEFINE LOOKUP TABLE FOR BONUS PER PERIOD

lookup_period_file = r"inputs/LookUp_Period.csv"

lookup_period = pd.read_csv(
    lookup_period_file,
    sep=",",          # `delimiter` y `sep` son equivalentes; elige uno
    encoding="utf-8",
    decimal=".",      # parsea decimales con punto
    thousands=",",    # parsea separador de miles con coma
    quotechar='"',
    index_col=0
)

# Clip y reasignar
lookup_period = lookup_period*0

lookup_period

,Bonus/Mile,4 Periods
Period,,
Night,0.0,
AM-Early,0.0,
AM-Peak,0.0,
AM-Shoulder,0.0,
MD,0.0,
PM-Shoulder,0.0,
PM-Peak,0.0,
PM-Late,0.0,


In [415]:
### DEFINE SEGMENT PARAMETERS
# Default configuration for time periods in traffic data

#TBD: Make this automatically
# period_template = [                 # (Period, Hours/Day, Peak/OP, 4Periods tag)
#     ("Night",        7, "OP",   "NT"),
#     ("AM-Early",     1, "OP",   "AM"),
#     ("AM-Peak",      3, "Peak", "AM"),
#     ("AM-Shoulder",  1, "OP",   "AM"),
#     ("MD",           5, "OP",   "MD"),
#     ("PM-Shoulder",  1, "OP",   "PM"),
#     ("PM-Peak",      3, "Peak", "PM"),
#     ("PM-Late",      3, "OP",   "PM"),
# ]

period_template = [                 # (Period, Hours/Day, Peak/OP, 4Periods tag)
    ("0",1, "OP","NT","Night"),
    ("1",1, "OP","NT","Night"),
    ("2",1, "OP","NT","Night"),
    ("3",1, "OP","NT","Night"),
    ("4",1, "OP","NT","Night"),
    ("5",1, "OP","NT","Night"),
    ("6",1, "OP","AM","AM-Early"),
    ("7",1, "Peak","AM","AM-Peak"),
    ("8",1, "Peak","AM","AM-Peak"),
    ("9",1, "Peak","AM","AM-Peak"),
    ("10",1, "OP","AM", "AM-Shoulder"),
    ("11",1, "OP","MD","MD"),
    ("12",1, "OP","MD","MD"),
    ("13",1, "OP","MD","MD"),
    ("14",1, "OP","MD","MD"),
    ("15",1, "OP","MD","MD"),
    ("16",1, "OP","PM","PM-Shoulder"),
    ("17",1, "OP","PM","PM-Peak"),
    ("18",1, "OP","PM","PM-Peak"),
    ("19",1, "OP","PM","PM-Peak"),
    ("20",1, "OP","PM","PM-Late"),
    ("21",1, "OP","PM","PM-Late"),
    ("22",1, "OP","PM","PM-Late"),
    ("23",1, "OP","NT","Night")

]

rows = []
years = [2025]

# Default time periods list (for reference)
default_time_periods = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

# Create the base scenario: hour -> time period mapping
hour_to_period = {
    0: "Night",
    1: "Night",
    2: "Night",
    3: "Night",
    4: "Night",
    5: "Night",
    6: "AM-Early",
    7: "AM-Peak",
    8: "AM-Peak",
    9: "AM-Peak",
    10: "AM-Shoulder",
    11: "MD",
    12: "MD",
    13: "MD",
    14: "MD",
    15: "MD",
    16: "PM-Shoulder",
    17: "PM-Peak",
    18: "PM-Peak",
    19: "PM-Peak",
    20: "PM-Late",
    21: "PM-Late",
    22: "PM-Late",
    23: "Night"
}

period_to_period = {
    'Evening': 'PM-Late',
    'Evening': 'Night',
    'EarlyAM': 'AM-Early',
    'AM': 'AM-Peak',
    'AM': 'AM-Shoulder',
    'Midday': 'MD',
    'PM': 'PM-Shoulder',
    'PM': 'PM-Peak'
}

# Define the segments and their parameters

peak_factor = 1 # 1.05 # Peak factor for adjustment at peak hour traffic

hov_percentage = pd.DataFrame({
    'Year' : [2025],
    'HOV percentage' : [0]
})

hov_percentage.set_index('Year', inplace=True)

# Define segment parameters base
seg_params = pd.DataFrame({
    'SegDir':   ["1NB","1SB","2NB","2SB","3NB","3SB","4NB","4SB","5NB","5SB","6NB","6SB","7NB","7SB","8NB","8SB","9NB","9SB","10NB","10SB"],
    'Length':    [1.3,1.3,1.3,1.3,0.5,0.5,1.6,1.6,2,2,3.6,3.6,2.9,2.9,3.8,3.8,3.4,3.4,4.5,4.5],
    'Inscope':   [0.8]*20,
    'Lanes_GP':  [4]*20, #
    'Lanes_ML':  [2]*20, # Lanes_ML': [2,2,2,2,2,2,2,2,3,3,2,2,2,2], # Do test changing segment 5
    'CapPerLane_GP': [2000]*20,
    'CapPerLane_ML': [1800]*20,
    'Speed_GP':  [55]*6 + [65]*2 + [70]*12,
    'Speed_ML':  [70]*20,
    'Alpha_GP':  [1]*20,
    'Beta_GP':   [6]*20,
    'Alpha_ML':  [1]*20,
    'Beta_ML':   [6]*20,
    'Min_Toll_2016': [None]*20,
    'Max_Toll_2016': [None]*20,
    'LanesGP_AM_Peak': [5]*20,
    'LanesGP_PM_Peak': [5]*20,
})

seg_params.set_index('SegDir', inplace=True)

# Compute capacities as lanes * cap per lane
seg_params['Cap_GP'] = seg_params['Lanes_GP'] * seg_params['CapPerLane_GP']
seg_params['Cap_ML'] = seg_params['Lanes_ML'] * seg_params['CapPerLane_ML']

# Compute peak capacities as Alpha * base capacity
seg_params['CapGP_Peak'] = seg_params['Alpha_GP'] * seg_params['Cap_GP']
seg_params['CapML_Peak'] = seg_params['Alpha_ML'] * seg_params['Cap_ML']

# Optional: if you want integer capacities
seg_params[['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']] = seg_params[
    ['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']
].astype(int)

# Preview
seg_params

,Length,Inscope,Lanes_GP,Lanes_ML,CapPerLane_GP,CapPerLane_ML,Speed_GP,Speed_ML,Alpha_GP,Beta_GP,Alpha_ML,Beta_ML,Min_Toll_2016,Max_Toll_2016,LanesGP_AM_Peak,LanesGP_PM_Peak,Cap_GP,Cap_ML,CapGP_Peak,CapML_Peak
SegDir,,,,,,,,,,,,,,,,,,,,
1NB,1.3,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
1SB,1.3,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
2NB,1.3,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
2SB,1.3,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
3NB,0.5,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
3SB,0.5,0.8,4,2,2000,1800,55,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
4NB,1.6,0.8,4,2,2000,1800,65,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
4SB,1.6,0.8,4,2,2000,1800,65,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600
5NB,2.0,0.8,4,2,2000,1800,70,70,1,6,1,6,None,None,5,5,8000,3600,8000,3600


In [416]:
import numpy as np

def adjusted_cumprod(row, target_year, multiplier):
    years = row.index
    print(row.values)
    factors = 1 + row.values
    
    # Find the index of the target year
    target_idx = list(years).index(target_year)
    
    # Apply multiplier to the target year's factor
    factors[target_idx] *= multiplier
    
    # Calculate cumulative product
    return pd.Series(np.cumprod(factors), index=years)

In [417]:
### HERE ARE THE MEASURED SPEEDS WE USE AS REFERENCE

measured_speeds_file = r"inputs/measured_speeds.csv"

measured_speeds_aux = pd.read_csv(
    measured_speeds_file,
    sep=",",          # `delimiter` y `sep` son equivalentes; elige uno
    encoding="utf-8",
    decimal=".",      # parsea decimales con punto
    thousands=",",    # parsea separador de miles con coma
    quotechar='"',
    index_col=0
)

# Convert the dictionary into a Series indexed like the rows of your df
period_index = pd.Series(hour_to_period)

# Assign period names to each row using the hour index
df_period = measured_speeds_aux.copy()
df_period["Period"] = df_period.index.map(period_index)

# Group by period and average
measured_speeds = df_period.groupby("Period").mean()

measured_speeds_aux

,1NB,2NB,3NB,4NB,5NB,6NB,7NB,8NB,9NB,10NB,1SB,2SB,3SB,4SB,5SB,6SB,7SB,8SB,9SB,10SB
0,59.149856,65.728754,65.865456,64.391916,66.241638,60.032992,59.188223,63.829398,63.647064,65.376095,57.092352,66.228092,60.868315,58.412998,64.616170,63.795808,63.591161,66.330597,67.213606,68.290323
1,59.149856,64.651233,65.865456,65.562678,66.241638,60.032992,59.835089,63.251756,63.647064,65.122699,58.678250,66.228092,60.868315,58.412998,64.616170,63.795808,63.591161,65.396363,66.517092,67.467548
2,59.724126,64.651233,65.865456,64.391916,65.650194,60.032992,60.496250,63.251756,63.965299,65.376095,58.678250,66.228092,60.868315,58.412998,64.054291,63.795808,62.864405,64.788025,65.498973,66.664363
3,59.149856,64.651233,65.865456,64.391916,66.241638,60.032992,60.832341,63.829398,64.611413,66.148253,59.226645,66.228092,60.868315,58.412998,64.616170,63.795808,63.225695,64.788025,65.498973,66.400867
4,60.309657,65.728754,65.865456,65.562678,68.081683,61.139591,60.832341,65.627409,66.630520,67.206625,59.226645,67.331894,60.868315,58.412998,65.187995,63.490565,63.960877,66.016234,66.174220,67.197678
5,60.309657,66.842800,65.865456,66.776801,69.366243,65.679659,58.870007,66.883436,68.048191,69.428332,60.354772,68.473113,64.249888,59.370588,65.770031,65.046706,65.875874,67.291910,67.213606,68.290323
6,53.492043,66.842800,53.889918,63.262233,63.937581,61.139591,61.863397,63.539264,64.286733,69.142619,59.226645,67.331894,64.249888,60.360098,66.965849,66.681046,65.483755,67.618570,68.651330,69.707134
7,32.376763,24.194633,29.639455,51.513533,41.777396,33.759011,21.512419,25.743348,58.683761,68.299416,57.611373,67.331894,18.653193,59.370588,65.187995,65.046706,64.334918,66.016234,67.567361,69.133413
8,20.103219,13.278536,35.926612,33.700442,29.529405,21.643216,11.489844,15.618590,60.919333,67.748614,53.254210,66.228092,13.293080,57.485807,63.502098,62.888759,62.864405,63.029074,66.863535,68.569059
9,17.626318,12.362775,22.369400,24.868602,27.232673,24.798675,19.483668,27.037985,62.711078,66.673239,54.631474,66.228092,12.994359,57.485807,61.901205,62.592114,62.864405,64.788025,66.863535,68.290323


In [418]:
### IMPORT GROWTHS FOR EACH CLASS
file_path_growths = r"inputs/growths_per_segment.csv"
base_growth_df = pd.read_csv(
    file_path_growths,
    delimiter=',',
    encoding='utf-8',
    decimal='.',        # ← this tells pandas how to parse decimals
    thousands=',',       # ← this tells pandas how to parse thousands
    quotechar='"'
)

base_growth_df = base_growth_df.iloc[:, 1:]
project_years = base_growth_df.columns[1:].tolist()
base_growth_df.iloc[:, 1:] =  base_growth_df.iloc[:, 1:] + 1

base_growth_df.loc[:, '2032'] *= 1.12

base_growth_df

,SegmentMapped,2026,2027,2028,2029,2030,2031,2032,2033,2034,...,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071
0,S1,1.018928,1.018928,1.018928,1.018928,1.018928,1.022899,1.145647,1.022899,1.022899,...,1.014078,1.014078,1.014078,1.014078,1.008447,1.008447,1.008447,1.008447,1.008447,1.008447
1,S2,1.025583,1.025583,1.025583,1.025583,1.025583,1.031514,1.155296,1.031514,1.031514,...,1.016660,1.016660,1.016660,1.016660,1.009996,1.009996,1.009996,1.009996,1.009996,1.009996
2,S3,1.023025,1.023025,1.023025,1.023025,1.023025,1.028939,1.152412,1.028939,1.028939,...,1.015544,1.015544,1.015544,1.015544,1.009326,1.009326,1.009326,1.009326,1.009326,1.009326
3,S4,1.022150,1.022150,1.022150,1.022150,1.022150,1.027914,1.151263,1.027914,1.027914,...,1.015180,1.015180,1.015180,1.015180,1.009108,1.009108,1.009108,1.009108,1.009108,1.009108
4,S5,1.018077,1.018077,1.018077,1.018077,1.018077,1.022963,1.145719,1.022963,1.022963,...,1.013537,1.013537,1.013537,1.013537,1.008122,1.008122,1.008122,1.008122,1.008122,1.008122
5,S6,1.018314,1.018314,1.018314,1.018314,1.018314,1.023435,1.146247,1.023435,1.023435,...,1.013597,1.013597,1.013597,1.013597,1.008158,1.008158,1.008158,1.008158,1.008158,1.008158
6,S7,1.020046,1.020046,1.020046,1.020046,1.020046,1.025430,1.148481,1.025430,1.025430,...,1.014297,1.014297,1.014297,1.014297,1.008578,1.008578,1.008578,1.008578,1.008578,1.008578
7,S8,1.019537,1.019537,1.019537,1.019537,1.019537,1.025049,1.148055,1.025049,1.025049,...,1.014060,1.014060,1.014060,1.014060,1.008436,1.008436,1.008436,1.008436,1.008436,1.008436
8,S9,1.017924,1.017924,1.017924,1.017924,1.017924,1.022940,1.145693,1.022940,1.022940,...,1.013443,1.013443,1.013443,1.013443,1.008066,1.008066,1.008066,1.008066,1.008066,1.008066
9,S10,1.020342,1.020342,1.020342,1.020342,1.020342,1.024674,1.147635,1.024674,1.024674,...,1.014523,1.014523,1.014523,1.014523,1.008714,1.008714,1.008714,1.008714,1.008714,1.008714


In [419]:
### IMPORT COUNTS AND SEPARATE BY CLASS AND PERIODS

file_path_counts = r"inputs/counts_by_hour_grouped_sorted.csv"
base_counts_df = pd.read_csv(
    file_path_counts,
    delimiter=',',
    encoding='utf-8',
    decimal='.',        # ← this tells pandas how to parse decimals
    thousands=',',       # ← this tells pandas how to parse thousands
    quotechar='"'
)

# --- Ajustar direcciones ---
base_counts_df["Direction"] = base_counts_df["Direction"].replace({"EB": "EB", "WB": "WB"})

# --- Crear columna Seg/Dir ---
base_counts_df["Seg/Dir"] = base_counts_df["Segment"].astype(str) + base_counts_df["Direction"]

# --- Función para procesar cada clase ---
def process_class(df_class):
    # Convertir a formato largo
    df_long = df_class.melt(
        id_vars=["Seg/Dir", "Segment", "Direction", "Class"],
        value_vars=[str(h) for h in range(24)],
        var_name="Hour",
        value_name="Volume"
    )
    
    # Mapear hora a periodo
    df_long["Hour"] = df_long["Hour"].astype(int)
    df_long["Period"] = df_long["Hour"].map(hour_to_period)
    
    # Agregar por Segment/Direction/Class/Period
    df_period = df_long.groupby(
        ["Seg/Dir", "Segment", "Direction", "Class", "Period"], as_index=False, sort=False
    ).agg({"Volume": "mean"}).round(0)
    
    # Pivot a formato ancho (periodos como columnas)
    period_order = df_period['Period'].unique()
    df_wide = df_period.pivot(
        index=["Seg/Dir", "Segment", "Direction", "Class"],
        columns="Period",
        values="Volume"
    )[period_order].reset_index()
    
    # Mantener solo Seg/Dir como índice
    df_proc = df_wide.drop(columns=["Class", "Direction", "Segment"]).set_index("Seg/Dir")
    
    return df_proc

# --- Separar por clases y procesar ---
dfs_by_class = {}
for cls in base_counts_df["Class"].unique():
    df_cls = base_counts_df[base_counts_df["Class"] == cls].copy()
    dfs_by_class[cls] = process_class(df_cls)


'''
Vehicle Classifications follow FHWA standards:
Lights: FHWA Classes 1-3 [Light Duty Vehicles]
Medium A: Classes 4-5 [Buses and Single Unit 2 axles trucks] 
Medium B: Class 6-7 [Single Unit 3 or 4 axles Trucks]
Heavy A: Classes 8-10 [Single Trailer 3 or more axles trucks]
Heavy B: Classes 11-13 [Combination Trucks Multitrailer Trucks]
'''

# --- Ejemplo de uso ---
df_lights = dfs_by_class["Lights"]
df_mediumA = dfs_by_class["Medium A"]
df_mediumB = dfs_by_class["Medium B"]
df_heavyA = dfs_by_class["Heavy A"]
df_heavyB = dfs_by_class["Heavy B"]


def extract_class(df, class_name):

    hour_cols = [str(i) for i in range(24)]
    df_class = df[df["Class"] == class_name].copy()
    df_class = df_class.set_index("Seg/Dir")[hour_cols]
    df_class.columns.name = "Period"
    return df_class

df_lights_hour    = extract_class(base_counts_df, "Lights")
df_mediumA_hour      = extract_class(base_counts_df, "Medium A")
df_mediumB_hour      = extract_class(base_counts_df, "Medium B")
df_heavyA_hour    = extract_class(base_counts_df, "Heavy A")
df_heavyB_hour    = extract_class(base_counts_df, "Heavy B")

df_heavyA_hour

Period,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Seg/Dir,,,,,,,,,,,,,,,,,,,,,
S1SB,260.0,231.0,253.0,233.0,317.0,334.0,298.0,297.0,344.0,414.0,...,388.0,323.0,277.0,292.0,329.0,371.0,339.0,297.0,272.0,244.0
S1NB,215.0,249.0,234.0,303.0,283.0,292.0,286.0,208.0,303.0,433.0,...,356.0,295.0,303.0,287.0,287.0,307.0,303.0,302.0,245.0,239.0
S2SB,121.0,108.0,104.0,97.0,123.0,163.0,171.0,161.0,175.0,251.0,...,191.0,171.0,130.0,160.0,170.0,129.0,152.0,122.0,115.0,104.0
S2NB,95.0,110.0,100.0,133.0,111.0,137.0,141.0,101.0,186.0,181.0,...,197.0,155.0,145.0,150.0,132.0,161.0,146.0,121.0,95.0,103.0
S3SB,179.0,156.0,183.0,177.0,238.0,251.0,215.0,225.0,302.0,343.0,...,306.0,262.0,209.0,230.0,251.0,297.0,254.0,209.0,199.0,186.0
S3NB,157.5,160.5,145.0,193.5,204.0,191.0,161.0,156.0,223.0,353.5,...,237.5,222.5,212.5,193.0,199.0,189.5,200.0,206.5,185.5,174.5
S4SB,188.0,179.0,173.0,198.0,254.0,263.0,234.0,274.0,318.0,377.0,...,311.0,262.0,222.0,232.0,278.0,295.0,239.0,223.0,218.0,196.0
S4NB,174.0,194.0,165.0,229.0,267.0,248.0,203.0,212.0,344.0,422.0,...,307.0,284.0,258.0,234.0,231.0,230.0,237.0,237.0,211.0,207.0
S5SB,177.0,161.0,178.0,193.0,261.0,282.0,236.0,296.0,396.0,464.0,...,352.0,312.0,266.0,287.0,326.0,312.0,262.0,221.0,222.0,194.0


In [420]:
# --- Lista de periodos según tus columnas ---
period_cols = [str(i) for i in range(24)]
# Diccionario de dataframes por clase
class_dfs = {
    "Lights": df_lights_hour,
    "Medium A": df_mediumA_hour,
    "Medium B": df_mediumB_hour,
    "Heavy A": df_heavyA_hour,
    "Heavy B": df_heavyB_hour
}

projected_long_by_class = {}

for cls_name, df_class in class_dfs.items():
    df = df_class.copy()
    
    # Resetear índice Seg/Dir y extraer Segment y Direction
    df = df.reset_index()
    df["Segment"] = df["Seg/Dir"].str.extract(r"(\d+)")[0]    # solo los números
    df["Direction"] = df["Seg/Dir"].str.extract(r"([A-Z]+)")[0]  # solo las letras
    df["Class"] = cls_name
    
    # Melt usando las columnas de periodos
    df_long = df.melt(
        id_vars=["Seg/Dir","Segment","Direction","Class"],
        value_vars=period_cols,
        var_name="Period",
        value_name="AADT "+str(df["Class"][0])
    )
    
    # Normalizar SegDir (opcional)
    df_long["SegDir"] = df_long["Seg/Dir"].str.strip().str.upper().str.lstrip("S")
    
    projected_long_by_class[cls_name] = df_long

# Ejemplo: ver Lights
projected_long_lights_df = projected_long_by_class["Lights"]
projected_long_mediumA_df = projected_long_by_class["Medium A"]
projected_long_mediumB_df = projected_long_by_class["Medium B"]
projected_long_heaviesA_df = projected_long_by_class["Heavy A"]
projected_long_heaviesB_df = projected_long_by_class["Heavy B"]
projected_long_lights_df


,Seg/Dir,Segment,Direction,Class,Period,AADT Lights,SegDir
0,S1SB,1,S,Lights,0,1289.0,1SB
1,S1NB,1,S,Lights,0,896.0,1NB
2,S2SB,2,S,Lights,0,852.0,2SB
3,S2NB,2,S,Lights,0,830.0,2NB
4,S3SB,3,S,Lights,0,1116.0,3SB
...,...,...,...,...,...,...,...
475,S8NB,8,S,Lights,23,954.0,8NB
476,S9SB,9,S,Lights,23,1139.5,9SB
477,S9NB,9,S,Lights,23,825.0,9NB
478,S10SB,10,S,Lights,23,1036.0,10SB


In [421]:
rows = []

for year in years:
    for seg in seg_params.index:  # e.g., "1NB", "1SB", etc.
        seg_data = seg_params.loc[seg]
        # Extraer parte numérica y dirección
        seg_numeric = ''.join(filter(str.isdigit, seg))  # e.g., "10"
        direction = seg[len(seg_numeric):]       
        for p, hrs, peak, tag, periods in period_template:
            rows.append({
                "Year": year,
                "SegDir": seg,
                "Segment": seg_numeric,        
                "Direction": direction,     
                "Period": p,
                "Hours/Day": hrs,
                "Peak": peak,
                "4Periods": tag,
                "Classic Periods": periods,

                # Parámetros técnicos
                "Length": seg_data["Length"],
                "Speed GP": seg_data["Speed_GP"],
                "Capacity GP": seg_data["CapPerLane_GP"] * seg_data["Lanes_GP"],
                "Alpha GP": seg_data["Alpha_GP"],
                "Beta GP": seg_data["Beta_GP"],
                "Speed ML": seg_data["Speed_ML"],
                "Capacity ML": seg_data["CapPerLane_ML"] * seg_data["Lanes_ML"],
                "Alpha ML": seg_data["Alpha_ML"],
                "Beta ML": seg_data["Beta_ML"],
                "MinToll": 0.5,
                "MinCapture": 0
            })

# --- plantilla base ---
first_model_df = pd.DataFrame(rows)

# --- merge para todas las clases ---
for cls_name, df_proj in projected_long_by_class.items():
    proj_merge_df = df_proj[["SegDir", "Period", f"AADT {cls_name}"]].copy()
    proj_merge_df.rename(columns={"AADT": f"AADT {cls_name}"}, inplace=True)
    
    first_model_df = first_model_df.merge(
        proj_merge_df,
        on=["SegDir", "Period"],
        how="left"
    )

# --- 1. Reshape growths a formato largo ---
growths_long = base_growth_df.melt(
    id_vars="SegmentMapped",
    var_name="Year",
    value_name="AnnualGrowth"
).copy()
growths_long["Year"] = growths_long["Year"].astype(int)

# --- 2. Calcular crecimiento acumulado desde 2025 ---
# Ordenamos por año y aplicamos cumprod
growths_long = growths_long.sort_values(["SegmentMapped", "Year"])
growths_long["GrowthFactor"] = (growths_long["AnnualGrowth"]).groupby(growths_long["SegmentMapped"]).cumprod()

# Ahora GrowthFactor(y) = factor acumulado 2025→y

# --- 3. Preparar plantilla ---
fm = first_model_df.copy()
fm["Year"] = fm["Year"].astype(int)
fm["Segment"] = fm["Segment"].astype(str).str.replace(r"^S", "", regex=True)
fm["SegmentMapped"] = "S" + fm["Segment"].astype(str)

# --- 4. Merge GrowthFactor ---
fm = fm.merge(
    growths_long[["SegmentMapped", "Year", "GrowthFactor"]],
    on=["SegmentMapped", "Year"],
    how="left"
)

fm["GrowthFactor"] = fm["GrowthFactor"].fillna(1.0)

# --- 5. Aplicar GrowthFactor a todas las clases ---
for cls_name in projected_long_by_class.keys():
    col = f"AADT {cls_name}"
    if col in fm.columns:
        fm[col] = (fm[col].fillna(0) * fm["GrowthFactor"]).round(1)

# --- 6. Limpieza ---
fm = fm.drop(columns=["SegmentMapped"])   # opcional

first_model_df = fm

first_model_df


,Year,SegDir,Segment,Direction,Period,Hours/Day,Peak,4Periods,Classic Periods,Length,...,Alpha ML,Beta ML,MinToll,MinCapture,AADT Lights,AADT Medium A,AADT Medium B,AADT Heavy A,AADT Heavy B,GrowthFactor
0,2025,1NB,1,NB,0,1,OP,NT,Night,1.3,...,1,6,0.5,0,896.0,19.0,5.0,215.0,34.0,1.0
1,2025,1NB,1,NB,1,1,OP,NT,Night,1.3,...,1,6,0.5,0,595.0,36.0,4.0,249.0,37.0,1.0
2,2025,1NB,1,NB,2,1,OP,NT,Night,1.3,...,1,6,0.5,0,469.0,39.0,2.0,234.0,35.0,1.0
3,2025,1NB,1,NB,3,1,OP,NT,Night,1.3,...,1,6,0.5,0,700.0,39.0,2.0,303.0,37.0,1.0
4,2025,1NB,1,NB,4,1,OP,NT,Night,1.3,...,1,6,0.5,0,1612.0,44.0,11.0,283.0,22.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2025,10SB,10,SB,19,1,OP,PM,PM-Peak,4.5,...,1,6,0.5,0,3440.0,41.0,12.0,344.0,26.0,1.0
476,2025,10SB,10,SB,20,1,OP,PM,PM-Late,4.5,...,1,6,0.5,0,2648.0,35.0,3.0,339.0,17.0,1.0
477,2025,10SB,10,SB,21,1,OP,PM,PM-Late,4.5,...,1,6,0.5,0,1898.0,28.0,8.0,266.0,27.0,1.0
478,2025,10SB,10,SB,22,1,OP,PM,PM-Late,4.5,...,1,6,0.5,0,1617.0,14.0,3.0,227.0,17.0,1.0


In [422]:
first_model_df["Capacity GP"] = first_model_df.apply(
    lambda row: seg_params.loc[row["SegDir"], 'Cap_GP'],
    axis=1
)

first_model_df["B1"] = first_model_df.apply(
    lambda row: beta_vals[row["Year"]].loc[row["4Periods"], 1],
    axis=1
)

first_model_df["B2"] = first_model_df.apply(
    lambda row: beta_vals[row["Year"]].loc[row["4Periods"], 2],
    axis=1
)

# Here we load th value of the counts and we multiply the peak hour values by a constant
lights_w = 1

heavies_w = 3
heavies_w_toll = 3
heavies_w_vot = 3

medium_A_w = 3 # TBD: Maybe try 2.5 or 2.75 for every pce value
medium_A_w_toll = 3
medium_A_w_vot = 3

medium_B_w = 3
medium_B_w_toll = 3
medium_B_w_vot = 3

heavy_A_w = 3
heavy_A_w_toll = 5
heavy_A_w_vot = 3

heavy_B_w = 3
heavy_B_w_toll = 3
heavy_B_w_vot = 3


# 2. Compute TotalLights
first_model_df["TotalLights"] = first_model_df["AADT Lights"] 

first_model_df["TotalMediumA"] = first_model_df["AADT Medium A"]

first_model_df["TotalMediumB"] = first_model_df["AADT Medium B"]

first_model_df["TotalHeavyA"] = first_model_df["AADT Heavy A"]

first_model_df["TotalHeavyB"] = first_model_df["AADT Heavy B"]

first_model_df["TotalVeh"] = first_model_df.apply(
    lambda row: row["TotalLights"] + row["TotalMediumA"] + row["TotalMediumB"] + row["TotalHeavyA"] + row["TotalHeavyB"],
    axis=1
)

first_model_df["Corridor PCE pre-fix"] = first_model_df.apply(
    lambda row: row["TotalLights"] * lights_w + row["TotalMediumA"] * medium_A_w + row["TotalMediumB"] * medium_B_w + row["TotalHeavyA"] * heavy_A_w + row["TotalHeavyB"] * heavy_B_w,
    axis=1
)

first_model_df["Corridor PCE"] = first_model_df["Corridor PCE pre-fix"] # To anulate the suppression

first_model_df["HOV3"] = first_model_df.apply(
    lambda row: row["TotalLights"] * hov_percentage.loc[row['Year']]['HOV percentage'],
    axis=1
)

first_model_df["TotalVeh"] = first_model_df.apply(
    lambda row: row["TotalLights"] + row["TotalMediumA"] + row["TotalMediumB"] + row["TotalHeavyA"] + row["TotalHeavyB"],
    axis=1
)

first_model_df["InScopeLights"] = first_model_df.apply(
    lambda row: (row["TotalLights"] - row["HOV3"]) * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeMediumA"] = first_model_df.apply(
    lambda row: row["TotalMediumA"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeMediumB"] = first_model_df.apply(
    lambda row: row["TotalMediumB"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeHeavyA"] = first_model_df.apply(
    lambda row: row["TotalHeavyA"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeHeavyB"] = first_model_df.apply(
    lambda row: row["TotalHeavyB"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeVeh"] = first_model_df.apply(
    lambda row: row["TotalVeh"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

# first_model_df.to_csv('model_test.csv')

In [423]:
def get_speed(row):

    max_VC = 1.2  # TBD: check if we need to change this value
    ETC_discount = 0.15


    gp_pce = (
        row["TotalLights"] * lights_w + 
        row["TotalMediumA"] * medium_A_w +
        row["TotalMediumB"] * medium_B_w +
        row["TotalHeavyA"] * heavy_A_w +
        row["TotalHeavyB"] * heavy_B_w
    )

    speedGP =  row["Speed GP"] / (1 + row["Alpha GP"] * ((gp_pce / row["Capacity GP"]) ** row["Beta GP"]))

    timeGP = 60 * row["Length"] / speedGP

    gp_vc = gp_pce / row["Capacity GP"]

    return pd.Series([speedGP, timeGP], index=["Speed GP","Time GP"])

In [424]:
first_model_df[["Speed GP Real", "Time GP"]] = first_model_df.apply(
    get_speed, axis=1, result_type='expand'
)

first_model_df.to_csv('model_test.csv')

In [425]:
new_counts_file = 'new_counts_only_peaks'

def get_pce(row):

    measured_speed = measured_speeds_aux.loc[int(row['Period'])][row['SegDir']]

    gp_pce = (
        row["TotalLights"] * lights_w + 
        row["TotalMediumA"] * medium_A_w +
        row["TotalMediumB"] * medium_B_w +
        row["TotalHeavyA"] * heavy_A_w +
        row["TotalHeavyB"] * heavy_B_w
    )

    speed_rate = row['Speed GP'] / measured_speed

    if speed_rate < 1:

        measured_speed = row['Speed GP'] - 0.01

    gp_pce_aux = row["Capacity GP"] * ((row['Speed GP'] / (measured_speed * row['Alpha GP']) - 1 / row['Alpha GP']) ** (1 / row['Beta GP']))

    if row["Classic Periods"] == "PM-Peak" or row["Classic Periods"] == "AM-Peak" or row["Classic Periods"] == "AM-Shoulder" or row["Classic Periods"] == "PM-Shoulder":

        pce_factor = gp_pce_aux / gp_pce

        pce_factor = np.clip(pce_factor, a_min = 0.6, a_max = 1.4)
    
    else:
        pce_factor = gp_pce_aux / gp_pce

        pce_factor = np.clip(pce_factor, a_min = 0.9, a_max = 1)

    # pce_factor = gp_pce_aux / gp_pce

    # pce_factor = np.clip(pce_factor, a_min = 0.7, a_max = 1.3)

    return pce_factor

    

In [426]:
first_model_df["PCE Factor"] = first_model_df.apply(
    lambda row: get_pce(row),
    axis=1
)

# first_model_df.to_csv('model_test.csv')

In [427]:
pce_factor = first_model_df.pivot(
    index=["Period"],
    columns="SegDir",
    values="PCE Factor"
)
pce_factor.to_csv('inputs/pce_factors_hourly.csv')

counts_aux = base_counts_df.drop(['Segment', 'Direction','AADT'], axis=1).set_index('Seg/Dir')

pce_factor

df_counts = counts_aux.copy()
df_counts.index = df_counts.index.str.replace("^S", "", regex=True)

hour_cols = [c for c in df_counts.columns if c not in ["Class"]]

df_final = df_counts.copy()

for segdir in df_counts.index:
    for h in hour_cols:
        df_final.loc[segdir, h] = df_counts.loc[segdir, h] * pce_factor.loc[h, segdir]

df_final.to_csv('adjusted_counts.csv')

In [428]:
# period_map = {0: "Night", 1:"Night", ...}
period_map_series = pd.Series(hour_to_period)

dropped_columns = ['Segment', 'Direction', 'Class', 'AADT']

df_counts = base_counts_df.drop(columns=dropped_columns).groupby(['Seg/Dir']).sum()

df_counts.index = df_counts.index.str.lstrip('S')
       # your traffic counts dataframe
df_factors = cap_factor        # your period × segdir dataframe

hours = list(range(24))
output = df_counts.copy()

df_counts

for h in hours:
    period = period_map_series[h]           # "AM-Early", "MD", etc.
    factors = df_factors.loc[str(h)]        # row of segdir factors

    output[str(h)] = df_counts.apply(
        lambda row: row[str(h)] * factors[row.name],
        axis=1
    )

df_counts.to_csv('base_total_counts.csv')
output.to_csv(f'{new_counts_file}.csv')

In [429]:
first_model_df["TotalLights"] = first_model_df.apply(
    lambda row: row["TotalLights"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalMediumA"] = first_model_df.apply(
    lambda row: row["TotalMediumA"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalMediumB"] = first_model_df.apply(
    lambda row: row["TotalMediumB"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalHeavyA"] = first_model_df.apply(
    lambda row: row["TotalHeavyA"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalHeavyB"] = first_model_df.apply(
    lambda row: row["TotalHeavyB"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

In [430]:
def get_capacity(row):

    measured_speed = measured_speeds_aux.loc[int(row['Period'])][row['SegDir']]

    gp_pce = (
        row["TotalLights"] * lights_w + 
        row["TotalMediumA"] * medium_A_w +
        row["TotalMediumB"] * medium_B_w +
        row["TotalHeavyA"] * heavy_A_w +
        row["TotalHeavyB"] * heavy_B_w
    )

    speed_rate = row['Speed GP'] / measured_speed

    if speed_rate < 1:

        measured_speed = row['Speed GP'] - 0.01

    capacity = gp_pce / ((row['Speed GP'] / (measured_speed * row['Alpha GP']) - 1 / row['Alpha GP']) ** (1 / row['Beta GP']))

    capacity_factor = capacity / row["Capacity GP"]

    return capacity_factor

In [431]:
first_model_df["Capacity Factor"] = first_model_df.apply(
    lambda row: get_capacity(row),
    axis=1
)

cap_factor = first_model_df.pivot(
    index=["Period"],
    columns="SegDir",
    values="Capacity Factor"
)

# cap_factor.to_csv('inputs/capacity_factors.csv')

cap_factor

SegDir,10NB,10SB,1NB,1SB,2NB,2SB,3NB,3SB,4NB,4SB,5NB,5SB,6NB,6SB,7NB,7SB,8NB,8SB,9NB,9SB
Period,,,,,,,,,,,,,,,,,,,,
0,2.571151e-06,0.000033,0.389936,4.925028,0.017018,0.038504,0.009018,0.752633,0.000130,0.000056,0.000012,0.000095,2.217668e-06,0.000055,1.106828e-06,0.000022,1.444826e-05,0.000052,6.738351e-07,3.753956e-06
1,4.826548e-07,0.000005,0.179132,1.123006,0.002419,0.002580,0.002725,0.103507,0.025154,0.000008,0.000005,0.000015,6.766493e-07,0.000011,7.242284e-07,0.000006,8.386672e-07,0.000002,2.383980e-07,1.430660e-06
10,2.438972e+00,5.435488,0.031403,25.591282,0.019541,29491.285149,0.004736,0.060287,0.951402,0.799417,0.955512,2.331194,6.569617e-01,2.773922,3.342561e-01,1.495523,5.411039e-01,1.952065,9.970300e-01,1.120915e+00
11,3.038762e+00,8.561570,0.045895,0.940245,0.058452,76007.140357,0.012164,0.093630,8.675184,0.643053,1.347919,0.932934,1.097834e+00,3.656386,1.104381e+00,3.056774,9.862514e-01,1.600734,5.756942e-01,1.464536e+00
12,5.302445e+00,16.664402,0.046484,0.805394,1.624063,105611.068645,0.021453,0.113932,35.046905,4.040249,1.655526,8.368124,3.084602e+00,7.638004,3.381007e+00,5.724796,2.113095e+00,2.407529,1.458485e+00,1.923829e+00
13,5.476781e+00,200.335308,0.035309,0.775433,0.882154,155120.038466,0.159754,0.084544,38.433376,3.221470,7.314698,2.125618,4.352523e+00,11.053297,3.299181e+00,9.742373,2.197742e+00,5.368326,1.200983e+00,4.421607e+00
14,2.976622e+00,273.394759,0.041316,0.594042,0.685297,154162.510377,0.031113,0.079983,41.381995,2.719833,2.294041,2.702015,3.969850e+00,9.035939,3.411856e+00,10.454765,1.457768e+00,4.180335,7.122680e-01,1.020512e+01
15,7.422847e+00,532.226315,0.008912,0.054889,0.176398,1.721014,0.006725,0.005984,8.379468,0.003855,2.127565,0.180620,3.482267e+00,3.755815,3.950306e+00,9.948466,2.448472e+00,4.859589,1.600258e+00,1.981150e+01
16,4.599115e+00,59.828607,0.004564,0.005379,0.100551,0.427420,0.000906,0.002836,4.015789,0.000655,6.148259,0.034830,3.007274e+00,0.534679,3.385577e+00,1.734228,2.447666e+00,0.818240,1.133300e+00,1.624652e+00


In [432]:
period_order = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

first_model_df["Total Corridor"] = first_model_df.apply(
    lambda row: row["Corridor PCE"] * row["Hours/Day"],
    axis=1
)


first_model_df["Period"] = pd.Categorical(
    first_model_df["Period"],
    categories=period_order,
    ordered=True
)

corridor_pce = first_model_df.pivot(
    index=["Period"],
    columns="SegDir",
    values="Total Corridor"
)

# corridor_pce.to_csv('corridor_vals.csv')

ValueError: Index contains duplicate entries, cannot reshape